In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
from conct_db import enginex
from datetime import date, datetime 

Conect succesfull


**LEER LOS DATOS DE ORIGIEN //CARTERA DE REGISTROS//**

In [2]:
file = pd.read_excel('z://PLANEAMIENTO/02-VENTAS/30-BEES/Cartera Registros.xlsx', skipfooter=3, sheet_name='Export')
file1 = pd.DataFrame(file)
file1.to_csv('z://PLANEAMIENTO/02-VENTAS/30-BEES/Cartera Registros.csv', encoding='UTF-8')
# file.sample()

In [3]:
data1 = pd.read_csv('z://PLANEAMIENTO/02-VENTAS/30-BEES/Cartera Registros.csv', encoding='UTF-8')
df = pd.DataFrame(data1)
# df.loc[[2134]]
# df.sample(3)
df.tail()

,Unnamed: 0,Código Distribuidor,Distribuidor,POC,Clientes,Registrado,Fecha registro,Adherible,Adherido,Fecha adhesion,Fecha Ult.Canje,CanjeL3M,Puntos Disponibles
9159,9159,445625,TOTAL LITORAL S.A EN FORMACION,44562500987625,1,1.0,2021-12-18,1.0,1.0,2021-12-18,2022-08-09,SI,2883.0
9160,9160,445625,TOTAL LITORAL S.A EN FORMACION,44562500987633,1,1.0,2021-10-28,1.0,1.0,2021-12-20,2022-09-22,SI,1670.0
9161,9161,445625,TOTAL LITORAL S.A EN FORMACION,44562500987638,1,1.0,2021-11-19,1.0,1.0,2021-11-19,2022-10-22,SI,646.0
9162,9162,445625,TOTAL LITORAL S.A EN FORMACION,44562500987645,1,0.0,NaN,0.0,NaN,NaN,NaN,NO,NaN
9163,9163,445625,TOTAL LITORAL S.A EN FORMACION,44562500999111,1,1.0,2022-02-01,1.0,1.0,2022-02-01,2022-10-14,SI,2478.0


In [4]:
df.columns

Index(['Unnamed: 0', 'Código Distribuidor', 'Distribuidor', 'POC', 'Clientes',
       'Registrado', 'Fecha registro', 'Adherible', 'Adherido',
       'Fecha adhesion', 'Fecha Ult.Canje', 'CanjeL3M', 'Puntos Disponibles'],
      dtype='object')

**TRANSFORMAR LOS DATOS PARA PODER CARGARLOS A LA BASE**

In [5]:
df['POC'] = df['POC'].astype(str)
# df['Fecha adhesion'] = df['Fecha adhesion'].convert_dtypes(int)
try:
    df.rename({' Fecha adhesion':'Fecha adhesion'}, axis=1, inplace=True)
except:
    pass
df['Fecha adhesion'] = pd.to_datetime(df['Fecha adhesion'],  format='%Y-%m-%d', errors='coerce')
df['Fecha registro'] = pd.to_datetime(df['Fecha registro'], format='%Y-%m-%d', errors='coerce')
# df['Fecha adhesion'] = df['Fecha adhesion'].dt.strftime('%m/%d/%Y')
df.tail()
# df.columns

,Unnamed: 0,Código Distribuidor,Distribuidor,POC,Clientes,Registrado,Fecha registro,Adherible,Adherido,Fecha adhesion,Fecha Ult.Canje,CanjeL3M,Puntos Disponibles
9159,9159,445625,TOTAL LITORAL S.A EN FORMACION,44562500987625,1,1.0,2021-12-18,1.0,1.0,2021-12-18,2022-08-09,SI,2883.0
9160,9160,445625,TOTAL LITORAL S.A EN FORMACION,44562500987633,1,1.0,2021-10-28,1.0,1.0,2021-12-20,2022-09-22,SI,1670.0
9161,9161,445625,TOTAL LITORAL S.A EN FORMACION,44562500987638,1,1.0,2021-11-19,1.0,1.0,2021-11-19,2022-10-22,SI,646.0
9162,9162,445625,TOTAL LITORAL S.A EN FORMACION,44562500987645,1,0.0,NaT,0.0,NaN,NaT,NaN,NO,NaN
9163,9163,445625,TOTAL LITORAL S.A EN FORMACION,44562500999111,1,1.0,2022-02-01,1.0,1.0,2022-02-01,2022-10-14,SI,2478.0


**METODO ALTERNATIVO PARA EXTRAER COD_DISTRI Y COD_CLIENTE**
# df['COD_CLIENTE'] = df.POC.str[-5:]
# df['COD_DISTRI'] = df.POC.str.extract('(\d{6,6}(?:\.\d+)?)')

In [6]:
df['COD_CLIENTE'] = df.POC.str.extract('(0{2,}\d+)')
df['COD_CLIENTE'] = df['COD_CLIENTE'].astype('Int64')
df.rename({'Código Distribuidor':'COD_DISTRI'}, axis=1, inplace=True)
# df = df.fillna()
df[['Registrado', 'Adherible', 'Adherido']] = df[['Registrado', 'Adherible', 'Adherido']].astype('Float64')
df[['Registrado', 'Adherible', 'Adherido']] = df[['Registrado', 'Adherible', 'Adherido']].astype('Int64')
try:
    df.drop(columns=['Unnamed: 0'], inplace=True)
except:
    pass
try:
    df.drop(columns=['POC', 'Distribuidor', 'Clientes'], inplace=True)
except:
    pass
df = df[['COD_DISTRI', 'COD_CLIENTE', 'Registrado', 'Fecha registro', 'Adherible', 'Adherido', 'Fecha adhesion', 'CanjeL3M', 'Puntos Disponibles']]
df.tail()

,COD_DISTRI,COD_CLIENTE,Registrado,Fecha registro,Adherible,Adherido,Fecha adhesion,CanjeL3M,Puntos Disponibles
9159,445625,987625,1,2021-12-18,1,1,2021-12-18,SI,2883.0
9160,445625,987633,1,2021-10-28,1,1,2021-12-20,SI,1670.0
9161,445625,987638,1,2021-11-19,1,1,2021-11-19,SI,646.0
9162,445625,987645,0,NaT,0,<NA>,NaT,NO,NaN
9163,445625,999111,1,2022-02-01,1,1,2022-02-01,SI,2478.0


**IMPORTAR A LA BASE**

In [7]:

try:
    df.to_sql('BEES_Cartera_Registros', con=enginex, if_exists='replace', index=False)
    # df.to_excel('z://PLANEAMIENTO/02-VENTAS/30-BEES/Cartera Registros.xlsx')
    print('Carga Exitosa ' + datetime.now().strftime('%A, %d of %B %Y a las %I:%M:%S'))
except Exception as ex:
    print(ex)
    

Carga Exitosa Wednesday, 02 of November 2022 a las 10:35:32
